# ULMFiT

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_11 import *

## Data

We will use the IMDB dataset.

In [ ]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [ ]:
path.ls()

[PosixPath('/home/ubuntu/.fastai/data/imdb/test'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/tmp_clas'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/README'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/unsup'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/train'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/tmp_lm'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/home/ubuntu/.fastai/data/imdb/ld.pkl')]

An ItemList that will read the texts in the corresponding filenames.

In [ ]:
#export
def read_file(fn): 
    with open(fn, 'r', encoding = 'utf8') as f: return f.read()
    
class TextList(ItemList):
    @classmethod
    def from_files(cls, path, extensions=None, recurse=True, include=None, **kwargs):
        if extensions is None: extensions = {'.txt'}
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, i):
        if isinstance(i, Path): return read_file(i)
        return i

In [ ]:
il = TextList.from_files(path, include=['train', 'test', 'unsup'])

In [ ]:
len(il.items)

100000

For text classification, we will split by the grand parent folder as before, but for language modeling, we take all the texts and just put 10% aside.

In [ ]:
#export
@classmethod
def _split_by_rand_pct(cls, il, pct=0.2):
    rand_idx = np.random.permutation(range(len(il.items)))
    cut = int(pct * len(il.items))
    train, valid = il.new(il[rand_idx[cut:]]),il.new(il[rand_idx[:cut]])
    return cls(train, valid)

SplitData.split_by_rand_pct = _split_by_rand_pct

In [ ]:
sd = SplitData.split_by_rand_pct(il, pct=0.1)

When we do language modeling, we will infer the labels from the text during training, so there's no need to label. The training loop expects labels however, so we need to add dummy ones. However there is a lot to do in terms of preprocessing the inputs!

### Tokenizing

We need to tokenize the dataset first. We will use a processor for this, in conjunction with the [spacy library](https://spacy.io/).

In [ ]:
#export
import spacy,html

Before even tokenizeing, we will apply a bit of preprocessing on the texts to clean them up:

In [ ]:
#export
BOS, EOS, UNK, PAD, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxbox", "xxeos", "xxunk", "xxpad", "xxrep", "xxwrep", "xxup", "xxmaj"

def sub_br(t):
    "Replaces the <br /> by \n"
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    return re_br.sub("\n", t)

def spec_add_spaces(t):
    "Add spaces between special characters"
    return re.sub(r'([/#])', r' \1 ', t)

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return re.sub(' {2,}', ' ', t)

def replace_rep(t):
    "Replace repetitions at the character level"
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
    
def replace_wrep(t):
    "Replace word repetitions"
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fixup(x):
    "List of replacements from html strings"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))
    
default_pre_rules = [fixup, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces, sub_br]
default_spec_tok = [BOS, UNK, PAD, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [ ]:
#export
def replace_all_caps(x):
    "Replace tokens in ALL CAPS in `x` by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

def deal_caps(x):
    "Replace all Capitalized tokens in `x` by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return res

def add_eos_bos(x): return [BOS] + x + [EOS]

default_post_rules = [deal_caps, replace_all_caps, add_eos_bos]

In [ ]:
#export
from spacy.symbols import ORTH

class TokenizeProcessor(Processor):
    def __init__(self, lang="en", chunksize=5000, pre_rules=None, post_rules=None): 
        self.chunksize = chunksize
        self.tokenizer = spacy.blank(lang)
        for w in default_spec_tok:
            self.tokenizer.tokenizer.add_special_case(w, [{ORTH: w}])
        self.pre_rules  = default_pre_rules  if pre_rules  is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules
    
    def process(self, items): 
        toks = []
        for i in progress_bar(range(0, len(items), self.chunksize)):
            chunk = items[i: i+self.chunksize]
            chunk = [compose(t, self.pre_rules) for t in chunk]
            docs = [[d.text for d in doc] for doc in self.tokenizer.tokenizer.pipe(chunk)]
            docs = [compose(t, self.post_rules) for t in docs]
            toks += docs
        return toks
    
    def proc1(self, item): 
        text = compose(item, self.pre_rules)
        toks = list(self.tokenizer.tokenizer(text))
        return compose(toks, self.post_rules)
    
    def deprocess(self, toks): return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):    return " ".join(tok)

### Numericalizing

In [ ]:
#export
import collections

class NumericalizeProcessor(Processor):
    def __init__(self, vocab=None, max_vocab=60000, min_freq=2): 
        self.vocab,self.max_vocab,self.min_freq = vocab,max_vocab,min_freq
    
    def process(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            freq = Counter(p for o in items for p in o)
            self.vocab = [o for o,c in freq.most_common(self.max_vocab) if c >= self.min_freq]
            for o in reversed(default_spec_tok):
                if o in self.vocab: self.vocab.remove(o)
                self.vocab.insert(0, o)
            self.otoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.vocab)})  
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi[o] for o in item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

In [ ]:
ll = label_by_func(sd, lambda x: 0, proc_x = [TokenizeProcessor(), NumericalizeProcessor()])

In [ ]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [ ]:
ll = pickle.load(open(path/'ld.pkl', 'rb'))

### Batching

We have a bit of work to convert our `LabelList` in a `DataBunch` as we don't just want batches of IMDB reviews. We want to stream through all the texts concatenated. We also have to prepare the targets that are the newt words in the text.

In [ ]:
#export
class LanguageModelPreLoader():
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data,self.bs,self.bptt,self.shuffle = data,bs,bptt,shuffle
        total_len = sum([len(t) for t in data.x])
        self.n_batch = total_len // bs
        self.batchify()
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def batchify(self):
        texts = self.data.x
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([tensor(t) for t in texts])
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

In [ ]:
dl = DataLoader(LanguageModelPreLoader(ll.valid, shuffle=True), batch_size=64)

In [ ]:
iter_dl = iter(dl)
x1,y1 = next(iter_dl)
x2,y2 = next(iter_dl)

In [ ]:
x1.size(),y1.size()

(torch.Size([64, 70]), torch.Size([64, 70]))

In [ ]:
vocab = ll.train.x.processors[1].vocab

In [ ]:
" ".join(vocab[o] for o in x1[0])

"xxbox xxmaj when watching a xxmaj bug 's xxmaj life for the first time in a long while , i could n't help but see the comparisons with last year 's xxmaj happy xxmaj feet . xxmaj as far as the main storyline goes , they are very similar , an outcast doing what he can to fit in while also attempting to be special . xxmaj it just goes"

In [ ]:
" ".join(vocab[o] for o in x2[0])

"to show you how much better that film could have been without its liberal diatribe conclusion . a lot of people disagree with me when i say that i really like xxmaj pixar 's sophomore effort . xxmaj sure it does n't manage to capture the splendor of xxmaj toy xxmaj story , nor is the animation out of this world . xxmaj however , the story is top -"

In [ ]:
" ".join(vocab[o] for o in y1[0])

"xxmaj when watching a xxmaj bug 's xxmaj life for the first time in a long while , i could n't help but see the comparisons with last year 's xxmaj happy xxmaj feet . xxmaj as far as the main storyline goes , they are very similar , an outcast doing what he can to fit in while also attempting to be special . xxmaj it just goes to"

In [ ]:
#export
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(LanguageModelPreLoader(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(LanguageModelPreLoader(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs))

In [ ]:
#export
def lm_databunchify(sd, bs, bptt, **kwargs):
    dls = get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs)
    return DataBunch(*dls)

In [ ]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

batchify
batchify


## AWD-LSTM

### LSTM

In [ ]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weight_ih = nn.Parameter(torch.randn(4 * hidden_size, input_size))
        self.weight_hh = nn.Parameter(torch.randn(4 * hidden_size, hidden_size))
        self.bias_ih = nn.Parameter(torch.randn(4 * hidden_size))
        self.bias_hh = nn.Parameter(torch.randn(4 * hidden_size))

    def forward(self, input, state):
        hx, cx = state
        #One big multiplication for all the gates is better than 4 smaller ones
        gates = (torch.mm(input, self.weight_ih.t()) + self.bias_ih +
                 torch.mm(hx, self.weight_hh.t()) + self.bias_hh)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)

        return hy, (hy, cy)

In [ ]:
class LSTMLayer(nn.Module):
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)

    def forward(self, input, state):
        inputs = input.unbind(1)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state

In [ ]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(64, 300),torch.zeros(64, 300))

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

106 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

25.9 ms ± 5.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
lstm = nn.LSTM(300, 300, 1, batch_first=True)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(1, 64, 300),torch.zeros(1, 64, 300))

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

103 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

The slowest run took 6.94 times longer than the fastest. This could mean that an intermediate result is being cached.
4.46 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Jit version

In [ ]:
import torch.jit as jit

In [ ]:
class LSTMCell(jit.ScriptModule):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weight_ih = nn.Parameter(torch.randn(4 * hidden_size, input_size))
        self.weight_hh = nn.Parameter(torch.randn(4 * hidden_size, hidden_size))
        self.bias_ih = nn.Parameter(torch.randn(4 * hidden_size))
        self.bias_hh = nn.Parameter(torch.randn(4 * hidden_size))

    @jit.script_method
    def forward(self, input, state):
        # type: (Tensor, Tuple[Tensor, Tensor]) -> Tuple[Tensor, Tuple[Tensor, Tensor]]
        hx, cx = state
        gates = (torch.mm(input, self.weight_ih.t()) + self.bias_ih +
                 torch.mm(hx, self.weight_hh.t()) + self.bias_hh)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)

        return hy, (hy, cy)

In [ ]:
class LSTMLayer(jit.ScriptModule):
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)

    @jit.script_method
    def forward(self, input, state):
        # type: (Tensor, Tuple[Tensor, Tensor]) -> Tuple[Tensor, Tuple[Tensor, Tensor]]
        inputs = input.unbind(1)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state

In [ ]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(64, 300),torch.zeros(64, 300))

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

99.1 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

9.61 ms ± 6.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Dropout

We want to use the AWD-LSTM from [Stephen Merity et al.](https://arxiv.org/abs/1708.02182). First, we'll need all different kinds of dropouts. Dropout consists into replacing some coefficients by 0 with probability p. To ensure that the averga of the weights remains constant, we apply a correction to the weights that aren't nullified of a factor `1/(1-p)`.

In [ ]:
#export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [ ]:
x = torch.randn(10,10)
dropout_mask(x, (10,10), 0.5)

tensor([[0., 2., 0., 0., 0., 0., 2., 2., 0., 0.],
        [0., 2., 2., 0., 0., 2., 0., 0., 0., 0.],
        [0., 2., 2., 2., 2., 2., 2., 2., 2., 0.],
        [0., 0., 2., 0., 0., 0., 2., 2., 2., 2.],
        [2., 2., 2., 2., 0., 2., 2., 2., 0., 2.],
        [2., 0., 0., 2., 2., 2., 0., 2., 0., 2.],
        [0., 2., 2., 0., 2., 2., 2., 0., 2., 0.],
        [2., 2., 2., 2., 0., 2., 0., 2., 2., 2.],
        [2., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
        [2., 2., 0., 2., 0., 2., 0., 2., 0., 0.]])

Once with have a dropout mask `m`, applying the dropout to `x` is simply done by `x = x * m`. We create our own dropout mask and don't rely on pytorch dropout because we want to nullify the coefficients on the batch dimension but not the token dimension (aka the same coefficients are replaced by zero for each word in the sentence). 

Inside a RNN, a tensor x will have three dimensions: seq_len, bs, vocab_size, so we create a dropout mask for the last two dimensions and broadcast it to the first dimension.

In [ ]:
#export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (1, x.size(1), x.size(2)), self.p)
        return x * m

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[-0.7134, -0.6048,  1.2643, -0.9341, -0.2501, -0.0056,  0.1866],
          [-0.4437,  1.7985, -0.7956, -0.4690,  0.4242,  1.0599, -0.4946],
          [-1.9174, -0.7605, -1.1349, -1.5009, -0.2407,  1.3109, -0.6536]],
 
         [[ 1.9492, -0.7999, -0.4039, -0.5594,  0.7353, -0.4139,  0.0038],
          [ 0.0530,  0.6208, -0.5785,  1.8239,  0.6503,  2.7063,  0.5661],
          [-0.2640, -1.4700,  1.0777, -0.3447,  0.8811, -0.9785,  1.6043]],
 
         [[ 2.5354,  1.4483, -1.1526, -1.4399,  1.0762, -0.0907,  0.3593],
          [-0.3155, -0.2931,  0.1420, -1.3725, -0.3247, -0.3834,  0.1773],
          [ 0.1863,  0.1054, -1.6390, -1.6150, -0.5115,  0.6026, -0.9085]]]),
 tensor([[[-0.0000, -0.0000,  1.8062, -1.3345, -0.0000, -0.0080,  0.2666],
          [-0.6338,  2.5693, -1.1366, -0.0000,  0.6060,  1.5141, -0.7065],
          [-2.7391, -0.0000, -1.6213, -0.0000, -0.3439,  0.0000, -0.9337]],
 
         [[ 0.0000, -0.0000, -0.5770, -0.7991,  0.0000, -0.5913,  0.0054],
          [ 0

Dropout applied to the weights of the inner LSTM cell. This is a little hacky.

In [ ]:
#export
import warnings

class WeightDropout(nn.Module):
    def __init__(self, module, weight_p=[0.], layer_names=['weight_hh_l0']):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            self.module._parameters[layer] = F.dropout(w, p=self.weight_p, training=False)

    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)

    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[-0.2069, -0.5788],
        [ 0.3250,  0.6312],
        [-0.5829,  0.3406],
        [ 0.4297, -0.3929],
        [-0.0161, -0.1363],
        [-0.3015, -0.4602],
        [ 0.3054, -0.6747],
        [ 0.6560,  0.1837]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[-0.0000, -0.9646],
        [ 0.5417,  0.0000],
        [-0.9716,  0.0000],
        [ 0.7162, -0.6548],
        [-0.0268, -0.2271],
        [-0.0000, -0.7671],
        [ 0.5090, -1.1245],
        [ 1.0933,  0.0000]], grad_fn=<MulBackward0>)

Dropout applied to full rows of the embedding matrix.

In [ ]:
#export
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    
    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb,self.embed_p = emb,embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000, -0.0000,  0.0000, -0.0000],
        [-2.3382, -2.7992, -2.6438, -1.5783,  1.0955,  2.6736,  0.2996],
        [ 0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.0000, -0.0000],
        [ 1.5681,  4.1461,  1.7590,  0.9174, -1.9847,  0.7747, -1.0808],
        [-0.0000, -0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
        [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
        [-0.0945, -2.5567, -3.0340,  2.5431, -2.0452, -0.6777,  2.3181]],
       grad_fn=<EmbeddingBackward>)

### Main model

In [ ]:
#export
def to_detach(h):
    "Detaches `h` from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [ ]:
#export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_pt=0.5):
        super().__init__()
        self.bs,self.emb_sz,self.n_hid,self.n_layers = 1,emb_sz,n_hid,n_layers
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz), 1,
                             batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        bs,sl = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        raw_output = self.input_dp(self.encoder_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output) 
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

In [ ]:
#export
class LinearDecoder(nn.Module):
    "To go on top of an AWD-LSTM module"
    initrange=0.1
    
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1]).contiguous()
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [ ]:
#export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [ ]:
#export
def get_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, tie_weights=True, bias=True, 
                       output_p=0.4, hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
    "To create a full AWD-LSTM"
    rnn_enc = AWD_LSTM(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                       hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [ ]:
tst_model = get_language_model(500, 20, 100, 2, 0)
tst_model = tst_model.cuda()

In [ ]:
x = torch.randint(0, 500, (10,5)).long()
z = tst_model(x.cuda())

In [ ]:
len(z)

3

### Callbacks to train the model

In [ ]:
#export
class GradientClipping(Callback):
    def __init__(self, clip=None): self.clip = clip
    def after_backward(self):
        if self.clip:  nn.utils.clip_grad_norm_(self.run.model.parameters(), self.clip)

In [ ]:
#export
class RNNTrainer(Callback):
    def __init__(self, alpha, beta): self.alpha,self.beta = alpha,beta
    
    def after_pred(self):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = self.pred[1],self.pred[2]
        self.run.pred = self.pred[0]
    
    def after_loss(self):
        #AR and TAR
        if self.alpha != 0.:  self.run.loss += self.alpha * self.out[-1].float().pow(2).mean()
        if self.beta != 0.:
            h = self.raw_out[-1]
            if len(h)>1: self.run.loss += self.beta * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
                
    def begin_epoch(self):
        #Shuffle the texts at the beginning of the epoch
        if hasattr(self.dl.dataset, "batchify"): self.dl.dataset.batchify()

In [ ]:
#export
def cross_entropy_flat(input, target):
    bs,sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

def accuracy_flat(input, target):
    bs,sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [ ]:
emb_sz, nh, nl = 300, 300, 1
model = get_language_model(len(vocab), emb_sz, nh, nl, 0, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback,
       Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, alpha=2., beta=1.),
       ProgressCallback]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt)

In [ ]:
learn.fit(1)

## Export

In [ ]:
!python notebook2script.py 12_text.ipynb

Converted 12_text.ipynb to exp/nb_12.py
